# MCP with Sqlite3

In [7]:
import sqlite3
from mcp.server.fastmcp import FastMCP

In [ ]:
mcp = FastMCP(name="mcp_with_sqlite3")

In [12]:
DB_NAME="demo.db"

In [13]:
@mcp.tool()
def add_data(query: str) -> bool:
    """Execute a SQL query to insert a record."""
    try:
        conn = sqlite3.connect(DB_NAME)
        conn.execute(query)
        conn.commit()
        conn.close()
        return True
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        return False

Tool already exists: add_data


In [14]:
@mcp.tool()
def read_data(query: str) -> list:
    """Execute a SQL query to read records."""
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.execute(query)
        results = cursor.fetchall()
        conn.close()
        return results
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        return []

Tool already exists: read_data


In [11]:
if __name__ == "__main__":
    print("Starting MCP server with SQLite3 support...")

Starting MCP server with SQLite3 support...


In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os 
from dotenv import load_dotenv

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GEMINI_API_KEY)

In [28]:
llm.invoke("hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f9a2e3ce-948a-4db7-9d6d-bdc2186275ec-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [29]:
SYSTEM_PROMPT = """
You are an AI assistant for Tool calling. Before helping the user, work with the tools to interact with the database.
"""

In [ ]:
from langchain.agents import initialize_agent, AgentType

In [34]:
async def get_agent():
    """Initialize the agent with the tools and system prompt."""
    return initialize_agent(
        tools=[add_data, read_data],
        llm=llm,
        name="mcp_agent",
        description="An agent that interacts with a SQLite3 database using MCP.",
        system_prompt=SYSTEM_PROMPT,
        verbose=True
    )

In [35]:
async def handle_user_messages(message: str, agent, agent_context):
    async for chunk in agent.astream(message):
        print(chunk, end="", flush=True)

In [36]:
from mcp.client.fastmcp import FastMCPClient

# Replace with your MCP server address and port
client = FastMCPClient(server_url="http://localhost:8000")

# Example: Call a tool on the server
result = client.call_tool("read_data", {"query": "SELECT * FROM your_table"})
print(result)

ModuleNotFoundError: No module named 'mcp.client.fastmcp'